In [ ]:
import time
import torch
import torch.nn.functional as F
import logging
from copy import copy

from data.utils import CNFFormula
from models.rbm import clauseRBM
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(level=logging.INFO)

def send_for_unit_propagation(assignments: torch.Tensor, variances: torch.Tensor) -> torch.Tensor:
    """
    Prepare a partial assignment for unit propagation by 'freezing' 
    the half of the variables with the highest variance and setting 
    the other half to -1 (unassigned).

    Parameters
    ----------
    assignments : torch.Tensor
        Current variable assignments (batch_size, num_vars), with values in {0, 1}.
    variances : torch.Tensor
        Estimated variances for each variable (batch_size, num_vars).

    Returns
    -------
    torch.Tensor
        Modified assignments where low-variance variables are set to -1.
    """
    batch_size, num_vars = assignments.shape
    # Determine how many variables to keep fixed
    half = num_vars // 2
    modified = assignments.clone()

    for b in range(batch_size):
        variance = variances[b]
        # Sort by descending variance
        sorted_indices = torch.argsort(variance, descending=True)
        # Set lower half to -1 (unassigned)
        modified[b, sorted_indices[half:]] = -1

    return modified

def fetch_unit_prop_result(formula: CNFFormula, partial_assignments: torch.Tensor) -> torch.Tensor:
    """
    Apply unit propagation to the given partial assignments using the CNFFormula directly.

    Parameters
    ----------
    formula : CNFFormula
        The formula object containing clauses as a list of lists of integers.
    partial_assignments : torch.Tensor
        A (batch_size, num_vars) tensor representing partially unassigned assignments.
        -1 indicates an unassigned variable, and 0/1 are assigned values.

    Returns
    -------
    torch.Tensor
        A (batch_size, num_vars) tensor with updated assignments after unit propagation.
    """
    results = []
    for assignment in partial_assignments.cpu():
        new_assignment = assignment.clone().detach()
        changed = True
        while changed:
            changed = False
            for clause in formula.clauses:
                num_unassigned = 0
                last_unassigned_lit = None
                clause_satisfied = False
                for lit in clause:
                    var = abs(lit) - 1  # 0-based index
                    val = new_assignment[var].item()
                    if val == -1:
                        num_unassigned += 1
                        last_unassigned_lit = lit
                    else:
                        is_true = (val == 1 and lit > 0) or (val == 0 and lit < 0)
                        if is_true:
                            clause_satisfied = True
                            break
                if not clause_satisfied and num_unassigned == 1:
                    # Unit clause, assign the last unassigned variable to satisfy the clause
                    var = abs(last_unassigned_lit) - 1
                    new_assignment[var] = 1 if last_unassigned_lit > 0 else 0
                    changed = True
        results.append(new_assignment)
    return torch.stack(results).to(partial_assignments.device)

def gather_and_count(assignments, clause_var_matrix, polarity):
    """
    Compute the clause satisfaction count for given assignments.
    """
    c = torch.einsum('bv,ckv->bck', assignments, clause_var_matrix) 
    return c, (((polarity + c) == 2) + ((polarity + c) == -1)).any(dim=-1).sum(dim=-1)


def time_remaining(start_time, time_limit):
    return (time.time() - start_time) <= time_limit


def construct_Q(clauses, num_var, device):
    Q = torch.zeros(clauses.shape[0], clauses.shape[1], num_var, device=device)
    for clause_idx, clause in enumerate(clauses):
        for lit_idx, lit in enumerate(clause):
            if lit != 0:
                Q[clause_idx, lit_idx, abs(lit)-1] = 1.0
    return Q


def rbmsat(formula, W_c, b_c, batch_size, seed, time_limit, heuristic_inteval=100, heuristic_delay=1, alpha=0.1):
    """
    RBM-based SAT solver main loop.

    Parameters
    ----------
    formula : CNFFormula
        The CNF formula object.
    W_c : torch.Tensor
        Clause RBM weight matrix.
    b_c : torch.Tensor
        Clause RBM bias vector.
    batch_size : int
        Batch size for parallel runs.
    seed : int
        Random seed.
    time_limit : float
        Maximum allowed time in seconds.
    upp : int
        Unit propagation period.
    upw : int
        Weight for unit propagation steps.
    alpha : float
        Smoothing parameter for variance updates.

    Returns
    -------
    s_max_list : list
        A list of tuples (step, max_satisfied_clauses).
    best_assignment : torch.Tensor
        The best assignment found.
    """
    formula_padding_copy = copy(formula)
    formula_padding_copy.padding()
    clauses = torch.tensor(formula_padding_copy.clauses, device=device)
    num_vars = formula.num_vars

    torch.manual_seed(seed)
    assignments = torch.bernoulli(torch.ones(batch_size, num_vars, device=W_c.device)*0.5)
    variances = 0.25 * torch.ones_like(assignments)  # initial variance guess

    polarity = torch.sign(clauses)
    clause_var_matrix = construct_Q(clauses, num_vars, device)
    W = torch.einsum('ck,kh->ckh', polarity, W_c)
    b = b_c.repeat([clauses.shape[0], 1]) + torch.mm((1 - polarity) / 2, W_c)

    step, d = 1, -1
    s_max_list = []
    best_assignment = torch.ones(num_vars, device=W_c.device)
    s_max = 0
    start_time = time.time()

    # Using these variables to store partial states for unit propagation
    last_partial = assignments.clone()
    with torch.no_grad():
        while time_remaining(start_time, time_limit):

            if heuristic_inteval > 0 and d == 0:
                # print("use heuristic")
                # Fetch updated assignments from unit propagation
                up_assignments = fetch_unit_prop_result(formula, last_partial)
                combined = torch.vstack([assignments, up_assignments])
                _, s_u = gather_and_count(combined, clause_var_matrix, polarity)
                ranks = torch.argsort(s_u)
                assignments = combined[ranks[-batch_size:]]
                variances = torch.vstack([variances, variances])[ranks[-batch_size:]]

            if heuristic_inteval > 0 and step % heuristic_inteval == 0:
                d = heuristic_delay
                # Update partial assignments for unit propagation
                last_partial = send_for_unit_propagation(assignments, variances)

            c, s = gather_and_count(assignments, clause_var_matrix, polarity)
            s_max_step, idx_max = s.max(dim=0)
            if s_max_step.item() > s_max:
                s_max = s_max_step.item()
                best_assignment = assignments[idx_max].clone()
                s_max_list.append((step, s_max))
                logging.info(f"New s_max found {s_max} at step {step}")

            h_logits = b + torch.einsum('bck,ckh->bch', c, W)
            h = torch.bernoulli(torch.sigmoid(h_logits))
            ro = torch.sigmoid(torch.einsum('bch,ckh,ckv->bv', h, W, clause_var_matrix))
            variances = (1 - alpha) * variances + alpha * ro * (1 - ro)
            assignments = torch.bernoulli(ro)

            step, d = step + 1, max(d - 1, -1)
    
    s_max_list.append((step, s_max_list[-1][-1]))

    return s_max_list, best_assignment


if __name__ == "__main__":
    # Example usage
    with open("wcnfdata/scpcyc09_maxsat.wcnf", 'r') as f:
        cnf_str = f.read()

    formula = CNFFormula(cnf_str)
    rbm = clauseRBM(formula.max_clause_length, device=device)

    batch_size = 128
    seed = 42
    timeout = 3000
    alpha, heuristic_inteval, heuristic_delay = 0.1, -1, 1

    s_max_list, best_assignment = rbmsat(formula, rbm.W, rbm.b, batch_size, seed, timeout, heuristic_inteval, heuristic_delay, alpha)
    logging.info(f"Best assignment found: {best_assignment}")

    # Optional: plot results
    if s_max_list:
        x_ax, y_ax = zip(*s_max_list)
        plt.step(x_ax, y_ax)
        plt.xlabel('Step')
        plt.ylabel('Max Satisfied Clauses')
        plt.show()

INFO:root:New s_max found 5522 at step 1
INFO:root:New s_max found 5581 at step 2
INFO:root:New s_max found 5596 at step 3
INFO:root:New s_max found 5606 at step 4
INFO:root:New s_max found 5615 at step 5
INFO:root:New s_max found 5631 at step 6
INFO:root:New s_max found 5649 at step 8
INFO:root:New s_max found 5651 at step 11
INFO:root:New s_max found 5653 at step 12
INFO:root:New s_max found 5657 at step 19
INFO:root:New s_max found 5661 at step 22
INFO:root:New s_max found 5669 at step 29
INFO:root:New s_max found 5671 at step 117
INFO:root:New s_max found 5673 at step 174
INFO:root:New s_max found 5674 at step 182
INFO:root:New s_max found 5675 at step 318
INFO:root:New s_max found 5676 at step 788
INFO:root:New s_max found 5679 at step 790
INFO:root:New s_max found 5681 at step 1141
INFO:root:New s_max found 5690 at step 2450
INFO:root:New s_max found 5691 at step 5415
INFO:root:New s_max found 5693 at step 78582


old version

In [ ]:
import os

bath_size = 128
seed = 42
timeout = 300

alpha, upp, upw = 0.1, 100, 1

best_num_satisfied_list = []
# test all files in ../wcnfdata
for file in os.listdir("./wcnfdata"):
    print(file)
    with open(os.path.join("./wcnfdata", file), 'r') as f:
        formula_str = f.read()
    
    formula = CNFFormula(formula_str)
    rbm = clauseRBM(formula.max_clause_length, device=device)
    s_max, best_v = rbmsat(formula, rbm.W, rbm.b, bath_size, seed, timeout, upp, upw, alpha)
    
    best_num_satisfied_list.append(s_max[-1][-1])

    print(f"{file}, {best_num_satisfied_list[-1]}")

brock200_3.clq.wcnf
new s_max found 803 at step 1
new s_max found 804 at step 397
new s_max found 805 at step 592
new s_max found 806 at step 766
new s_max found 807 at step 1029
new s_max found 811 at step 1560
new s_max found 812 at step 6315
new s_max found 813 at step 9824
new s_max found 815 at step 18707
new s_max found 817 at step 21471
new s_max found 819 at step 31340
new s_max found 821 at step 88583
new s_max found 822 at step 89522
brock200_3.clq.wcnf, 822
maxcut-140-630-0.7-12.wcnf
new s_max found 974 at step 1
new s_max found 993 at step 2
new s_max found 1005 at step 3
new s_max found 1011 at step 4
new s_max found 1012 at step 6
new s_max found 1015 at step 7
new s_max found 1016 at step 8
new s_max found 1021 at step 9
new s_max found 1022 at step 11
new s_max found 1025 at step 12
new s_max found 1027 at step 13
new s_max found 1030 at step 14
new s_max found 1032 at step 15
new s_max found 1037 at step 16
new s_max found 1040 at step 17
new s_max found 1041 at step 2

In [5]:
best_num_satisfied_list

[822,
 1093,
 1091,
 1093,
 1093,
 1092,
 1098,
 1093,
 1095,
 1091,
 1088,
 487,
 1333,
 1777,
 3777,
 5691]

In [ ]:
best_num_sat = [822, 1093, 1091, 1093, 1093, 1092, 1098, 1093, 1095, 1091, 1088, 487, 1333, 1777, 3777, 5691]
import os
from data.utils import CNFFormula
best_possible = []
# test all files in ../wcnfdata
for file in os.listdir("./wcnfdata"):
    print(file)
    with open(os.path.join("./wcnfdata", file), 'r') as f:
        formula_str = f.read()
    
    formula = CNFFormula(formula_str)
    best_possible.append(formula.num_clauses)

diff = [0] * len(best_num_sat)
for i in range(len(diff)):
    diff[i] = best_possible[i] - best_num_sat[i]

best_sol = [214, 167, 168, 167, 167, 168, 162, 165, 163, 167, 169, 7, 21, 30, 60, 823]
print(diff)

brock200_3.clq.wcnf
maxcut-140-630-0.7-12.wcnf
maxcut-140-630-0.7-20.wcnf
maxcut-140-630-0.7-28.wcnf
maxcut-140-630-0.7-4.wcnf
maxcut-140-630-0.7-45.wcnf
maxcut-140-630-0.7-6.wcnf
maxcut-140-630-0.8-1.wcnf
maxcut-140-630-0.8-18.wcnf
maxcut-140-630-0.8-26.wcnf
maxcut-140-630-0.8-39.wcnf
ram_k3_n11.ra0.wcnf
ram_k3_n14.ra0.wcnf
ram_k3_n15.ra0.wcnf
ram_k3_n18.ra0.wcnf
scpcyc09_maxsat.wcnf
[216, 167, 169, 167, 167, 168, 162, 165, 163, 167, 170, 8, 32, 43, 99, 1221]


In [ ]:
ours = [216, 167, 169, 167, 167, 168, 162, 165, 163, 167, 170, 8, 32, 43, 99, 1221]
satS = [214, 167, 169, 167, 167, 168, 162, 165, 163, 167, 169, 7, 21, 30, 60, 823, 1, 1, 1, 639, 984, 1, 1]
best = [214, 167, 168, 167, 167, 168, 162, 165, 163, 167, 169, 7, 21, 30, 60, 823, 1, 1, 1, 640, 986, 1, 1]

s_j = []
for j in range(len(ours)):
    s_j.append((best[j] + 1) / (satS[j] + 1))

score = sum(s_j) / len(s_j)
print(score)

0.9077161583166862


assignment test

In [ ]:
import torch
from data.utils import CNFFormula
from main import count_satisfied_clauses
v = torch.tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
        0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
        1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

# v_str = "111111101111111001111111111111010111111111100011011100111011101101110001111111101111011011111111011110011111111111101110110110011110010"

# v = torch.tensor([[int(i) for i in v_str]])

with open("wcnfdata/uaq-min-nr-nr50-np400-rpp5-nc0-rs0-t0-plb100-n9.wcnf", 'r') as f:
    cnf_str = f.read()
formula = CNFFormula(cnf_str)

best_v, best_num_satisfied = count_satisfied_clauses(
    formula, 
    v
)
print(best_num_satisfied)
print(best_v)

2682
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
         0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
         1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0.

In [12]:
v_str = "1 -2 -3 -4 -5 -6 7 -8 -9 -10 -11 12 -13 -14 -15 -16 17 -18 -19 20 -21 -22 23 -24 -25 -26 27 -28 -29 30 -31 -32 -33 -34 -35 -36 -37 -38 39 -40 -41 42 -43 -44 45 46 -47 48 49 -50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 -155 156 157 -158 159 160 161 162 163 164 165 166 167 -168 169 170 171 172 173 174 175 -176 177 178 179 180 181 182 183 184 185 -186 187 188 189 190 191 192 -193 194 195 196 197 198 199 200 -201 202 -203 204 205 206 -207 208 209 210 211 212 213 -214 215 -216 -217 218 219 220 221 222 223 -224 225 -226 227 228 229 230 231 232 233 234 235 -236 237 238 -239 -240 241 242 243 244 245 246 -247 248 249 250 251 252 253 254 255 256 257 258 259 260 -261 -262 -263 264 265 266 267 268 269 270 271 272 -273 274 275 276 277 278 279 280 281 -282 283 284 285 286 287 288 289 290 -291 292 -293 294 295 296 -297 -298 299 -300 301 302 303 304 305 306 307 -308 -309 310 311 312 -313 -314 -315 316 317 318 319 320 321 322 323 324 325 326 -327 328 329 330 -331 332 333 334 335 336 337 338 339 340 341 342 343 -344 345 346 347 348 349 350 351 352 -353 354 355 356 357 -358 359 -360 361 362 363 364 -365 -366 367 368 369 -370 -371 372 373 374 375 376 -377 -378 -379 380 381 382 383 384 385 386 387 -388 389 390 391 392 393 -394 -395 -396 -397 398 -399 400 401 402 403 404 405 406 407 408 409 410 411 -412 413 -414 415 416 417 418 419 420 -421 422 423 424 -425 426 427 428 429 430 431 -432 -433 434 -435 -436 437 438 439 -440 441 442 443 444 -445 -446 447 -448 -449 -450"
v_list = v_str.split()
v_temp = []
for var in v_list:
    if int(var) > 0:
        v_temp.append(1.0)
    else:
        v_temp.append(0.0)

v_torch = torch.tensor([v_temp])
print(v_torch)


tensor([[1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
         0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
         1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
         0., 1., 1., 1., 1.,

In [ ]:
from data.utils import CNFFormula
from main import solve_maxsat
import os

best_v_list = []
best_num_satisfied_list = []
# test all files in ../wcnfdata
for file in os.listdir("./wcnfdata"):
    print(file)
    with open(os.path.join("./wcnfdata", file), 'r') as f:
        formula_str = f.read()
    
    formula = CNFFormula(formula_str)
    best_v, best_num_satisfied = solve_maxsat(
        formula, 
        max_time=300, 
        heuristic_interval=100, 
        batch_size=1024)

    best_v_list.append(best_v)
    best_num_satisfied_list.append(best_num_satisfied)

    print(f"{file}, {best_num_satisfied}")

brock200_3.clq.wcnf
Step 100, Best: 813, current: 810, formula size: 1038, Elapsed Time: 16.84s
Step 200, Best: 818, current: 803, formula size: 1038, Elapsed Time: 29.70s
Step 300, Best: 818, current: 801, formula size: 1038, Elapsed Time: 44.76s
Step 400, Best: 818, current: 801, formula size: 1038, Elapsed Time: 60.85s
Step 500, Best: 818, current: 806, formula size: 1038, Elapsed Time: 77.98s
Step 600, Best: 818, current: 802, formula size: 1038, Elapsed Time: 97.43s
Step 700, Best: 818, current: 805, formula size: 1038, Elapsed Time: 110.33s
Step 800, Best: 818, current: 802, formula size: 1038, Elapsed Time: 123.68s
Step 900, Best: 818, current: 806, formula size: 1038, Elapsed Time: 136.62s
Step 1000, Best: 818, current: 803, formula size: 1038, Elapsed Time: 149.38s
Step 1100, Best: 818, current: 808, formula size: 1038, Elapsed Time: 162.05s
Step 1200, Best: 818, current: 796, formula size: 1038, Elapsed Time: 174.84s
Step 1300, Best: 818, current: 804, formula size: 1038, Ela

RuntimeError: einsum(): subscript h has size 7 for operand 1 which does not broadcast with previously seen size 8

rbm_model test

In [20]:
from models.rbm import clauseRBM
import torch

rbm_random = clauseRBM(5, F_s=-1.0, num_epochs=0, lr=0.01, device='cpu', verbose=True)
rbm_trained = clauseRBM(5, F_s=-1.0, num_epochs=20000, lr=0.01, device='cpu', verbose=True)

v_neg = torch.tensor([[0.0, 0.0, 0.0, 0.0, 0.0]])
v_pos1 = torch.tensor([[1.0, 0.0, 0.0, 0.0, 0.0]])
v_pos2 = torch.tensor([[1.0, 1.0, 0.0, 0.0, 0.0]])
v_pos3 = torch.tensor([[1.0, 1.0, 1.0, 0.0, 0.0]])
v_pos4 = torch.tensor([[1.0, 1.0, 1.0, 1.0, 0.0]])
v_pos5 = torch.tensor([[1.0, 1.0, 1.0, 1.0, 1.0]])

print(rbm_random.free_energy(v_neg).item(), rbm_random.free_energy(v_pos1).item(), rbm_random.free_energy(v_pos2).item(), rbm_random.free_energy(v_pos3).item(), rbm_random.free_energy(v_pos4).item(), rbm_random.free_energy(v_pos5).item())
print(rbm_trained.free_energy(v_neg).item(), rbm_trained.free_energy(v_pos1).item(), rbm_trained.free_energy(v_pos2).item(), rbm_trained.free_energy(v_pos3).item(), rbm_trained.free_energy(v_pos4).item(), rbm_trained.free_energy(v_pos5).item())

Saved pre-trained RBM to rbm_models\rbm_length_5_F_-1.0_num_epochs_0_lr_0.01_device_cpu.pth
Epoch 200, Loss: 0.40392351150512695
Epoch 400, Loss: 0.15035410225391388
Epoch 600, Loss: 0.07847076654434204
Epoch 800, Loss: 0.054054055362939835
Epoch 1000, Loss: 0.0438944511115551
Epoch 1200, Loss: 0.03873978182673454
Epoch 1400, Loss: 0.03555965796113014
Epoch 1600, Loss: 0.033251821994781494
Epoch 1800, Loss: 0.031401023268699646
Epoch 2000, Loss: 0.0298561193048954
Epoch 2200, Loss: 0.028566580265760422
Epoch 2400, Loss: 0.027515148743987083
Epoch 2600, Loss: 0.026690170168876648
Epoch 2800, Loss: 0.026074139401316643
Epoch 3000, Loss: 0.02563847415149212
Epoch 3200, Loss: 0.025342069566249847
Epoch 3400, Loss: 0.025134537369012833
Epoch 3600, Loss: 0.02496362291276455
Epoch 3800, Loss: 0.024781839922070503
Epoch 4000, Loss: 0.024546733126044273
Epoch 4200, Loss: 0.024215035140514374
Epoch 4400, Loss: 0.02373451739549637
Epoch 4600, Loss: 0.02303897961974144
Epoch 4800, Loss: 0.02205649